In [108]:
import tensorflow as tf
import json 
import numpy

simulations_path = '/Users/remydubois/Dropbox/Remy/Data/Simulations-1/smFISH_simulations__batch_0001.json'
cell_path = '/Users/remydubois/Dropbox/Remy/Data/cellLibrary.json'

In [44]:
with open(cell_path) as f:
    jsoncells = json.load(f)
with open('testjson.json', 'w+') as f:
    f.write("{'a':1}\n{'b':2}")
    # text = f.read()
# f = open(simulations_path)
# g = open(cell_path)

# ds = tf.data.Dataset.from_tensor_slices(g.read())
ds = tf.data.TextLineDataset(['testjson.json'])
# ds = ds.map(tf.as_string)
ds = ds.map(tf.py_func(lambda x: json.loads(x), ))  # Here four threads for preprocessing
# ds = ds.batch()
# or
# ds = ds.apply(tf.contrib.data.map_and_batch(
#     map_func=parse_fn, batch_size=FLAGS.batch_size, num_parallel_batches=4))  # Not sure perfectly equivalent
# ds = ds.prefetch(1)  # Allow consumer and producer to overlap their work, precise here the queue size.
# ds = ds.repeat(None)
# or 
# ds = tf.contrib.data.shuffle_and_repeat(ds)
it = ds.make_one_shot_iterator()
ex = it.get_next()
sess = tf.Session()
print(sess.run(ex))



TypeError: py_func() missing 2 required positional arguments: 'inp' and 'Tout'

In [40]:
x = tf.constant("{'a':1}")
y = tf.decode_json_example()
sess = tf.Session()
print(sess.run(x).decode())

AttributeError: 'Tensor' object has no attribute 'decode'

In [56]:
import tensorflow as tf

writer = tf.python_io.TFRecordWriter('/Users/remydubois/Desktop/testTFRecord.tfrecords')


def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


pos = [[0, 1], [1, 2]]
feature = {'positions': _bytes_feature(tf.compat.as_bytes(str(pos).encode())),
           'shape': tf.train.Feature(int64_list=tf.train.Int64List(value=[2]))
           }

example = tf.train.Example(features=tf.train.Features(feature=feature))

writer.write(example.SerializeToString())

writer.close()


In [57]:
data_path = '/Users/remydubois/Desktop/testTFRecord.tfrecords'  # address to save the hdf5 file
with tf.Session() as sess:
    feature = {'positions': tf.FixedLenFeature([], tf.string),
               'shape': tf.FixedLenFeature([], tf.int64)}
    
    # Create a list of filenames and pass it to a queue
    filename_queue = tf.train.string_input_producer([data_path], num_epochs=1)
    # Define a reader and read the next record
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    # Decode the record read by the reader
    features = tf.parse_single_example(serialized_example, features=feature)
    # Convert the image data from string back to the numbers
    image = tf.decode_raw(features['positions'], tf.int64)


Tensor("DecodeRaw_2:0", shape=(?,), dtype=int64)


In [154]:
import tensorflow as tf
import pickle

data = [[[1, 1], [2, 2]], [[0, 0], [1, 0], [1, 1]]]
labels = [0, 1]

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


with tf.python_io.TFRecordWriter('testTFRecord.tfrecord') as writer:
    for d, l in zip(data, labels):
        image_raw = numpy.array(d, dtype=numpy.int32).ravel().tostring()
        example = tf.train.Example(
            features=tf.train.Features(
                feature={
                    'image_raw': _bytes_feature(image_raw),
                    'label': _int64_feature(l)
                }))
        writer.write(example.SerializeToString())


def decode(serialized_example):
    """Parses an image and label from the given `serialized_example`."""
    features = tf.parse_single_example(
        serialized_example,
        # Defaults are not specified since both keys are required.
        features={
            'image_raw': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.int64)
        })

    # Convert from a scalar string tensor (whose single string has
    # length mnist.IMAGE_PIXELS) to a uint8 tensor with shape
    # [mnist.IMAGE_PIXELS].
    image = tf.decode_raw(features['image_raw'], tf.int32)
    out = tf.reshape(image, (-1, 2))
    
    label = tf.cast(features['label'], tf.int32)

    return out, label


ds = tf.data.TFRecordDataset('testTFRecord.tfrecord')
ds = ds.map(decode)
ds = ds.map(lambda x, l: (tf.sparse_to_dense(x, (tf.reduce_max(x) + 1, ) * 2, 1, validate_indices=False), l))
# ds = ds.map(lambda x: tf.pad(x, tf.substr([[5,5], [5,5]], tf.shape(x))))
it = ds.make_one_shot_iterator()
print(sess.run(it.get_next()))
print(sess.run(it.get_next()))

# dataset = tf.data.Dataset.from_tensor_slices(numpy.load('testmat.npy'))
# dataset = dataset.map(lambda x: tf.sparse_to_dense(x, (2, 2), 1))
# it = dataset.make_one_shot_iterator()
# print(sess.run(it.get_next()))

(array([[0, 0, 0],
       [0, 1, 0],
       [0, 0, 1]], dtype=int32), 0)


(array([[1, 0],
       [1, 1]], dtype=int32), 1)


In [ ]:
import luigi


def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


class TFRecordBuilder(luigi.task):
    def requires(self):
        return [Merge()]

    def output(self):
        return luigi.LocalTarget('PositionsTFRecord.tfrecord')

    def run(self):
        with tf.python_io.TFRecordWriter('testTFRecord.tfrecord') as writer, open(self.input()[0]) as fin:
            # Or read by chunk with pandas to not fit it all in memory
            js = json.load(fin)
            for item in js:
                d = item['positions']
                l = item['label']
                image_raw = numpy.array(d, dtype=numpy.int32).ravel().tostring()
                example = tf.train.Example(
                    features=tf.train.Features(
                        feature={
                            'image_raw': _bytes_feature(image_raw),
                            'label': _int64_feature(l)
                        }))
                writer.write(example.SerializeToString())


In [ ]:
import os
import luigi
import mmap
import pandas
from sklearn.preprocessing import LabelEncoder

class StackSimulations(luigi.Task):
    def requires(self):
        return []

    def output(self):
        return luigi.LocalTarget("stacked_file.json")

    def run(self):
        simulation_batches_path = [simulations_path + f for f in os.listdir(simulations_path) if
                                   f.lower().endswith('.json')]

        # Combine all the simulations
        simulations = []
        for b in simulation_batches_path:
            with open(b, 'r') as f:
                simulation_batch = json.load(f)
            simulations.extend(simulation_batch)
        dataset = pandas.DataFrame(simulations)
            
        dataset = dataset[['cell_ID', 'RNA_pos', 'pattern_name']]

        # Now shuffle
        dataset = dataset.sample(frac=1).reset_index(drop=True)

        dataset.to_json(simulations_path + 'stacked_file.json', orient='records', lines=True)


class Merge(luigi.task):
    def requires(self):
        return [StackSimulations()]
    
    def output(self):
        return luigi.LocalTarget("merged_file.json")
    
    def run(self):
        simulations = pandas.read_json(self.input()[0], orient='records', lines=True)
        jsoncells = pandas.read_json(cell_path)
        jsoncells['cell_ID'] = jsoncells.index
        merged = simulations.join(jsoncells, 'cell_ID')
        merged['pos'] = merged.pos_nuc + merged.pos_cell + merged.RNA_pos
        le = LabelEncoder()
        merged['pattern_name'] = le.fit_transform(merged['pattern_name'])
        merged = merged[['pos', 'pattern_name']]
        with self.output().open('w') as fout:
            merged.to_json(fout, orient='index')
        
        
        
